In [1]:
%load_ext autoreload

%autoreload 2

import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib
torch.set_default_dtype(torch.float64)

In [2]:
from models import SmallNet

torch.random.manual_seed(0)
model = SmallNet()
w0 = [w+0.0 for w in model.parameters()]
d = np.sum([w.numel() for w in model.parameters()])
print ("Total parameters",d)  

Total parameters 476


In [3]:
from utils import getMnistDataSubset
trainloader, trainset = getMnistDataSubset(2000)
for batch_idx, (data, target) in enumerate(trainloader):
  break
data, target = data.double(), target

# TRCG

In [ ]:
from TRCG import TRCG



torch.random.manual_seed(0)
model = SmallNet()


# this function is used to compute the loss. If the data-size is larger and it does not
# fit the GPU memory, it can split the computation on multiple parts and hence it can be
# called automatically in loops
def closureFun(part, total, device):
    scale = 1.0 / total
    if total > 1:
        size = data.size()[0]
        local_size = size // total
        inp = data[part*local_size:(part+1)*local_size:].to(device)
        out = target[part*local_size:(part+1)*local_size:].to(device)

    else:
        inp = data.to(device)
        out = target.to(device)
    
    
    output = model(inp)
    loss = scale*F.nll_loss(output, out)  # The negative log likelihood loss.   https://pytorch.org/docs/stable/nn.functional.html#nll-loss
#     grad_f = torch.autograd.grad(loss, model.parameters(), create_graph=True)  # computation of gradient 
#     print("closure",loss.item())
    return loss


radius = .1
device = 'cpu'
opt = TRCG(model, radius, device, closure_size = 1)

    

fvalHist = []
radiusHist=[]
for it in tqdm(range(40)):
    outFval, radius, cnt_compute, cg_iter  = opt.step(closureFun)
    fvalHist.append(outFval);radiusHist.append(radius)
plt.plot(fvalHist);plt.grid(True);plt.title('F(w)')
plt.show()
plt.plot(radiusHist);plt.grid(True);plt.title('Radius')

  0%|          | 0/40 [00:00<?, ?it/s]